Copyright 2023 Franco Aquistapace

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Import libraries and code

In [1]:
import pandas as pd
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

import time

In [2]:
from gaussian_metric import *
from tools import *

# Comparing integration methods

We need to find out what is the best way to integrate functions that take vectors as inputs. First, let's check that both implementations of our functions work properly.

In [3]:
# Build dummy configuration
sc_test = gen_sc_lattice(2,2,2,1)

# Build some positions
p1 = np.array([0,   0,   0  ])
p2 = np.array([0.5, 0,   0  ])
p3 = np.array([0,   0.5, 0  ])
p4 = np.array([0,   0.5, 0.5])
p5 = np.array([1,   1,   1  ])

# Print measure representation values
print('Vector implementation')
for p in [p1,p2,p3,p4,p5]:
    C_rep = measure_representation(p, sc_test)
    print("C(%.1f, %.1f, %.1f) = %.4f" % (*p, C_rep))
print('\nElement-wise implementation')
for p in [p1,p2,p3,p4,p5]:
    C_rep = measure_representation_exp(
        p[0],p[1],p[2], sc_test)
    print("C(%.1f, %.1f, %.1f) = %.4f" % (*p, C_rep))

Vector implementation
C(0.0, 0.0, 0.0) = 1.0056
C(0.5, 0.0, 0.0) = 0.4173
C(0.0, 0.5, 0.0) = 0.4173
C(0.0, 0.5, 0.5) = 0.1732
C(1.0, 1.0, 1.0) = 1.0056

Element-wise implementation
C(0.0, 0.0, 0.0) = 1.0056
C(0.5, 0.0, 0.0) = 0.4173
C(0.0, 0.5, 0.0) = 0.4173
C(0.0, 0.5, 0.5) = 0.1732
C(1.0, 1.0, 1.0) = 1.0056


This seems to be working, now we have to see which integration method is better. We already know that the integration performed by scipy is precise, but we can compare its performance to a more rudimentary integration method. This last method consists of sampling the integration interval, evaluating the function on said points and adding the results, multiplied by the sample volume, to get an estimate of the integral.

In [14]:
sc1 = gen_sc_lattice(2,2,2,1)
sc2 = gen_sc_lattice(2,2,2,1)
sc2 = displace(sc2, np.array([-1.5,-1.5,-1.5]))
margin = 2
step_size = 0.1

# Designing some configurations to test the metric